In [1]:
import re
import pandas as pd

df = pd.read_csv("SalahYunus.csv")

# 1 - GRADES
df.rename(columns={'السنة الدراسية': 'School Year'}, inplace=True)
categories = {
    'kg': r'k|كي|كى',
    'gradu': r'خري|جر|graduate|201[3-9]|202[0-4]|university|Becha|ريوس|جامع|كلي',
    'grads': r'grade|year|yr|الصف',
    'primary': r'بتد|pri|بر',
    'prepatory': r'داد|متو|pre|عداد',
    'sec': r'sec|ثانو|عامة',
}

numbers = {
    '1': r'1|وان|١|One|ول',
    '2': r'2|۲|تو|Two|secon|ثاني|تاني|٢ث',
    '3': r'3|۳|ثري|Three|ثالث|تالت|third',
    '4': r'4|٤|فور|Four| راب',
    '5': r'5|٥|فايف|Five|خام',
    '6': r'6|٦|سيكس|Six|ساد',
    '7': r'7|٧|سفن|Seven|ساب',
    '8': r'8|٧|ايت|Eight|ثام',
    '9': r'9|٩|ناين|Nine|التاس',
    '10': r'10|١٠|تن|Ten|العاش',
    '11': r'11|١١|الفن|Eleven|الحادي ع|حداشر',
    '12': r'12|١٢|تولف|Twelve|الثاني ع|اتناشر',
}

mapping = {
    'kg': {'1': 'kg1', '2': 'kg2', '3': 'kg3'},
    'primary': {str(i): f'Grade {i}' for i in range(1, 7)},
    'preparatory': {str(i): f'Grade {i+6}' for i in range(1, 4)},
    'secondary': {str(i): f'Grade {i+9}' for i in range(1, 4)},
    'grads': {str(i): f'Grade {i}' for i in range(1, 13)},
    'gradu': 'finished school',
}

def grades(values):
    value = values.lower()

    if re.search(categories['kg'], value):
        for num, num_pattern in numbers.items():
            if re.search(num_pattern, value):
                return f'kg{num}'
    
    elif re.search(categories['sec'], value):
        for num, num_pattern in numbers.items():
            if re.search(num_pattern, value):
                return f'Grade {int(num) + 9}'

    elif re.search(categories['primary'], value):
        for num, num_pattern in numbers.items():
            if re.search(num_pattern, value):
                return f'Grade {num}' if int(num) <= 6 else 'other'

    elif re.search(categories['prepatory'], value):
        for num, num_pattern in numbers.items():
            if re.search(num_pattern, value):
                return f'Grade {int(num) + 6}'

    elif re.search(categories['gradu'], value):
        return 'finished school'

    for num, num_pattern in numbers.items():
        if re.search(num_pattern, value):
            return f'Grade {num}'

    return 'other'


df["School Year"] = df["School Year"].apply(grades)




# 2 - Date & Time
df.rename(columns={'طابع زمني': 'timestamp'}, inplace=True)

df['time'] = pd.to_datetime(df['timestamp'])

df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df.pop('timestamp')
df.pop('time')


# 3 - Some translation
df.rename(columns={'هل ترى التعليم عن بعد مفيد ومهم؟': 'Is online learning crucial?'}, inplace=True)

translation_dict = {
    'نعم': 'Yes',
    'ربما': 'Maybe',
    'لا': 'No'
}
df['Is online learning crucial?'] = df['Is online learning crucial?'].map(translation_dict)

# 4 - rmve the ID column
df.drop('id', axis=1, inplace=True)


# 5 - Other Courses
df.rename(columns={'ما هي الخدمات التعليمية التي تود الاشتراك أو الاستفسار عنها؟': 'Other Courses'}, inplace=True)

df_dummies = df['Other Courses'].str.get_dummies(sep=', ')
df = pd.concat([df, df_dummies], axis=1)
df.drop('Other Courses', axis=1, inplace=True)
df.drop('Option 13', axis=1, inplace=True)
df.replace({1: 'tick'}, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Fill empty values
df.fillna("Unknown",inplace=True)

# 6 - Leading Source
df.rename(columns={'من أين علمت عن جينيس؟': 'Who told you about Genius?'}, inplace=True)

def replace_source(value):
    value = value.lower()
    social_media = [
        r'فيس|اجتماع|بوك|face|book|social|ميديا|نت|F.B',
    ]
    Relatives = [
        r'صحاب|صاحب|دكتور|أب|أم|family|friends|someone|شخاص|شخص',
    ]
    Ad = [
        r'اعلان|ممول|جينيس',
    ]

    for pattern in social_media:
        if re.search(pattern, value):
            return 'Social Media'

    for pattern in Relatives:
        if re.search(pattern, value):
            return 'Relatives'

    for pattern in Ad:
        if re.search(pattern, value):
            return 'Ad'
        
    return 'Unknown'

df["Who told you about Genius?"] = df["Who told you about Genius?"].astype(str)
df["Who told you about Genius?"] = df["Who told you about Genius?"].apply(replace_source)


# 7 - Countries
df.rename(columns={'مكان السكن': 'State'}, inplace=True)
df.rename(columns={'بلد الإقامة': 'Country'}, inplace=True)

countries = {
    'Egypt': r'زايد|امبابه|منيب|alexa|منوف|عمران|منصور|mansour|بدر|شبرا|سكندري|شرقية|سوهاج|sohag|egy|cair|october|giza|بلطيم|عين|دميا|معمور|شارع|شيخ|قنا|qena|قاهر|اسماعيل|رمضان|هرم|حلوان|فيوم|جيز|زقازيق|ارض|شرقي|بلبيس|مطري|نصر|تجمع|البلد|العبور|مسكن|سادات|سويف|حي|بور',
    'Saudi Arabia': r'السعودية|saudi arabia|r|arar|arabic|سعود|جد|رياض|riyad|رماية',
    'Kuwait': r'kuwait|الكويت|salmiya|سالمية|كويت',
    'Qatar': r'qatar|قطر',
    'United Arab Emirates': r'uae|united arab emirates|ابو ظبي|dubai',
    'Oman': r'oman|عمان',
    'Sweden': r'sweden|سويد',
}

def categorize_country(city):
    city = city.lower()

    for country, pattern in countries.items():
        if re.search(pattern, city):
            return country

    return 'Unknown'

df['Country'] = df['State'].apply(categorize_country)
df.pop('State')

translation_dict = {
    'الرياضيات ( الماتيفيك )': 'Mathematics (Matific)',
    'القارئ المثقف': 'The Educated Reader',
    'القاعدة النورانية': 'Nooraniyah Rule (Quranic Learning)',
    'الكُتّاب (تحفيظ قرآن)': 'Writers (Quran Memorization)',
    'المدرسة الاونلاين': 'Online School',
    'المستكشف الصغير': 'Little Explorer',
    'دبلومة البرمجة المتخصصة': 'Specialized Programming Diploma',
    'دبلومة الجرافكس': 'Graphics Diploma',
    'دبلومة الديجتال ماركتنج': 'Digital Marketing Diploma',
    'كورس اللغة الألمانية': 'German Language Course',
    'كورس اللغة الإنجليزية': 'English Language Course',
    'مبدائ البرمجة التأسيسية': 'Fundamentals of Programming'
}

df.rename(columns=translation_dict, inplace=True)


# FINALLY

df.to_csv("SalahYunus_Clean.csv")